# Hausarbeit im Seminar Textanalyse WiSe 22/23

Zunächst werden die Parteien definieiert und die Wahlprogramme (soweit vorhanden) eingelsen.

In [92]:
# import modules
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.snowball import GermanStemmer



In [103]:
PARTIES = [
    "ALL_BB",
    "GDF",
    "GHG",
    "JUSO",
    "LHG",
    "LISTE_VOLT",
    "NORDCAMPUS",
    "RCDS"
]

PROGRAMS = {}

for party in PARTIES:
    with open(f"{party}.txt", "r", encoding="utf-8") as f:
        file_content = f.read()
        PROGRAMS[party] = file_content

df = pd.DataFrame(PROGRAMS.items(), columns=["party", "program"] )

df

,party,program
0,ALL_BB,"test test test Email sent, waiting for response"
1,GDF,Hochschulpolitik - Nicht mehr und nicht wenige...
2,GHG,"Liebe Studis, \nEs ist wieder so weit, die Hoc..."
3,JUSO,"Liebe Wähler*innen,\nvor euch seht ihr unser W..."
4,LHG,AStA-Beiträge senken\nAllgemeine Studiengebühr...
5,LISTE_VOLT,"Volt & Die LISTE verstehen sich als linkes, pr..."
6,NORDCAMPUS,Mehr interdisziplinäre Zusammenarbeit\nViele S...
7,RCDS,Auslandsstudium\nWer ein Semester im Ausland v...


## Data Cleaning
Im nächsten Schritt werden die Wahlprogramme gereinigt. Das Bedeutet, dass einige, für die Analyse unwichtige oder störende, Elemente gelöscht werden. Darunter zählen:
- Zeilenumbrüche
- großbuchstaben (werden klein geschrieben)
- anderes

In [104]:
TO_REPLACE = {
    "&": "und",
    "*": "_"
}

# Alle Buchstaben klein schreiben
df.program = df.program.str.lower()

for e in TO_REPLACE:
    df.program = df.program.str.replace(e, TO_REPLACE[e])


# tokenize
for i, prog in df.program.items():
    df.program[i] = word_tokenize(prog, language="german")
# rerun cell above to fix data type if you get an error

df.program

C:\Users\User\AppData\Local\Temp\ipykernel_6592\2861057112.py:10: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.program = df.program.str.replace(e, TO_REPLACE[e])


0    [test, test, test, email, sent, ,, waiting, fo...
1    [hochschulpolitik, -, nicht, mehr, und, nicht,...
2    [liebe, studis, ,, es, ist, wieder, so, weit, ...
3    [liebe, wähler_innen, ,, vor, euch, seht, ihr,...
4    [asta-beiträge, senken, allgemeine, studiengeb...
5    [volt, und, die, liste, verstehen, sich, als, ...
6    [mehr, interdisziplinäre, zusammenarbeit, viel...
7    [auslandsstudium, wer, ein, semester, im, ausl...
Name: program, dtype: object

In [107]:
# stopwords definieren
custom_stopwords = ["test"]
stopwords_used = stopwords.words("german") + custom_stopwords

# stemming setup
stemmer = GermanStemmer()


for i, prog in df.program.items():
    # remove stopwords
    no_stopwords = [word for word in prog if word not in stopwords_used]

    # remove short tokens
    new_list = [word for word in no_stopwords if len(word) > 1]

    # stemming
    new_list = [stemmer.stem(word) for word in new_list]
    # TODO studierende wird nicht zu studi 

    # save to df
    df.program[i] = new_list




df.program

0                 [email, sent, waiting, for, respons]
1    [hochschulpolit, mehr, wenig, hochschulpolit, ...
2    [lieb, studis, weit, hochschulwahl, steh, 16.,...
3    [lieb, wahler_inn, seht, wahlprogramm, hochsch...
4    [asta-beitrag, senk, allgemein, studiengebuhr,...
5    [volt, list, versteh, link, progressiv, queer-...
6    [mehr, interdisziplinar, zusammenarbeit, viel,...
7    [auslandsstudium, wer, sem, ausland, verbringt...
Name: program, dtype: object

In [131]:
counts_series = pd.Series(df.program[1]).value_counts(normalize=True).mul(100).round(1)
counts_series
percent_cutoff = 0.5 # %
new_series = counts_series[counts_series > percent_cutoff]
new_series
#TODO fix stemming/lemmatization (ask)
#TODO think about research question

studier               2.2
digital               1.1
student               1.1
studium               1.1
ford                  0.9
mehr                  0.9
imm                   0.9
setz                  0.9
campus                0.9
geht                  0.8
bess                  0.8
universitat           0.7
we                    0.7
steht                 0.7
viel                  0.7
gemeinschaft          0.6
and                   0.6
bedeutet              0.6
sqm                   0.6
studierendenschaft    0.6
fachschaft            0.6
gemeinsam             0.6
dtype: float64